In [3]:
import numpy as np
import pandas as pd

import bebi103
import cmdstanpy
import arviz as az

import bokeh.plotting
import bokeh.io
from bokeh.io import output_file, save
bokeh.io.output_notebook()

/home/max/micromamba/envs/pol_stats/lib/python3.12/site-packages/bebi103/viz.py:38: UserWarning: DataShader import failed with error "cannot import name 'Callable' from 'collections' (/home/max/micromamba/envs/pol_stats/lib/python3.12/collections/__init__.py)".
Features requiring DataShader will not work and you will get exceptions.
  warnings.warn(


Loading BokehJS ...

$\dot{A}=gA$ 

$g(D)=g_{max}(D_{end}-D)/(D_{end}-D_{inflection})$ if $D>D_{inflection}$

$D(t)=\int_{t_0}^t C(A)$

Same parameters for conditions but different $A_0$

In [4]:
import numpy as np
from scipy.integrate import solve_ivp
import bokeh.plotting
import bokeh.io

# Constants for the new model
class ModelParameters:
    def __init__(self, C0=0.0, C1=0.3, D_decrease=5.0, g_max=0.1):
        self.C0 = C0
        self.C1 = C1
        self.D_decrease = D_decrease
        self.D_end = 10 + D_decrease  # Define D_end in terms of D_inflect and D_decrease
        self.g_max = g_max

# Define the Heaviside step function for g
def g(D, params):
    if D < 10:
        return params.g_max
    elif D < params.D_end:
        return (params.D_end - D) / (params.D_end - 10) * params.g_max
    return 0.0

# Define the function C(A)
def C(A, params):
    return params.C0 + params.C1 * A

# ODE system for the new model
def ode_system(t, y, params):
    A, D = y
    dA_dt = A * g(D, params)         # Differential equation for A
    dD_dt = C(A, params)             # dD/dt as per given integral
    return [dA_dt, dD_dt]

# Solve the model using initial conditions and parameters
def A_theor_model(t, A0, params, D0=0.0):
    solution = solve_ivp(
        ode_system,
        [t.min(), t.max()],
        [A0, D0],
        args=(params,),
        t_eval=t,
        method='RK45'
    )
    return solution.y[0]  # Returns A(t) solution

# Main function to simulate different realizations
def ppc(x, rng):
    params = ModelParameters(
        C0=np.abs(rng.normal(0, 0.15)),
        C1=rng.normal(0.15, 0.05),
        D_decrease=10**rng.normal(1.0, 0.2),
        g_max=10**rng.normal(-1.3, 0.3)
    )
    A0 = 10**rng.normal(0.2, 0.15)    # Initial value of A based on random normal distribution
    sigma = 0 * np.abs(rng.normal(0, 1.0))  # Set sigma to control noise level, if desired
    A_solution = A_theor_model(x, A0, params)
    return rng.normal(A_solution, sigma)

# Generate x values and compute multiple realizations
x = np.linspace(0, 96, 100)
rng = np.random.default_rng()
f_ppc = np.array([ppc(x, rng) for _ in range(100)])

# Plotting with Bokeh
p = bokeh.plotting.figure(
    x_axis_label="x",
    y_axis_label="f",
    frame_height=400,
    frame_width=400,
)

# Plot each realization
for f in f_ppc:
    p.line(x + 48, f, line_alpha=0.6)  # Add transparency for clarity

bokeh.io.show(p)


In [5]:
import os
Sahra_Shivani_path=(r'/media/max_kotz/sahra_shivani_data')
Output_path=os.path.join(Sahra_Shivani_path,'sorted_data')
Curv_Thick_path=os.path.join(Output_path,'for_curv_thick')
hdf5_file_path = os.path.join(Curv_Thick_path, 'scalarGrowthData.h5')

# Aoad the DataFrame from the HDF5 file
df = pd.read_hdf(hdf5_file_path, key='data')

p = bokeh.plotting.figure(
    frame_width=350,
    frame_height=250,
    x_axis_label="time in hpf",
    y_axis_label="Surface Area",
    #x_axis_type="log",
    toolbar_location='above',
)

colors = bokeh.palettes.Category10_5
items = []
print(df)
for color, (genotype, sub_df) in zip(colors, df.groupby("condition")):
    print(genotype)
    items.append(
        (
            bokeh.models.LegendItem(
                label=genotype,
                renderers=[
                    p.scatter(
                        sub_df["time in hpf"],
                        sub_df["Surface Area"]*1e-4,
                        color=color,
                    )
                ],
            )
        )
    )

p.add_layout(bokeh.models.Legend(items=items), 'right')
bokeh.io.show(p)

                             Mask Folder        Volume   Surface Area  \
0    20240409sox10_clau132hpfdev3_Stitch  2.156393e+06  121968.863980   
1    20240410sox10_clau144hpfdev1_Stitch  1.949042e+06  112923.567433   
2    20240103sox10_clau126hpfreg4_Stitch  2.121201e+06  102161.069891   
3    20240103sox10_clau126hpfreg3_Stitch  2.013371e+06   87418.650875   
4    20240329sox10_clau144hpfdev1_Stitch  1.920304e+06  103024.294823   
..                                   ...           ...            ...   
185     20240626sox10_clau120hpfreg1ff72  2.098723e+06   98366.658752   
186     20240626sox10_clau120hpfreg2ff72  2.280549e+06   91374.312464   
187     20240626sox10_clau120hpfreg3ff72  2.406311e+06  105994.062418   
188     20240626sox10_clau120hpfreg4ff72  2.390018e+06   94558.304416   
189     20240626sox10_clau120hpfreg6ff72  2.419007e+06  107340.797662   

     Integrated Thickness        L PD        L AP     condition  time in hpf  \
0            2.102532e+06  479.895636  339.

In [7]:
sm = cmdstanpy.CmdStanModel(stan_file='fit_Dose.stan')

15:24:27 - cmdstanpy - INFO - compiling stan file /home/max/Documents/01_Code/SahraPipline/3_2_fits/fit_Dose.stan to exe file /home/max/Documents/01_Code/SahraPipline/3_2_fits/fit_Dose
15:24:34 - cmdstanpy - INFO - compiled model executable: /home/max/Documents/01_Code/SahraPipline/3_2_fits/fit_Dose


In [8]:
def df_to_data(cat_Dev,cat_Reg):
    filtered_df = df[df['condition'] == cat_Reg]
    t_Reg = filtered_df['time in hpf'].to_numpy()
    A_Reg = filtered_df['Surface Area'].to_numpy()*1e-4
    sorted_indices = np.argsort(t_Reg)
    t_Reg = t_Reg[sorted_indices]
    A_Reg = A_Reg[sorted_indices]

    filtered_df = df[df['condition'] == cat_Dev]
    t_Dev = filtered_df['time in hpf'].to_numpy()
    A_Dev = filtered_df['Surface Area'].to_numpy()*1e-4
    sorted_indices = np.argsort(t_Dev)
    t_Dev = t_Dev[sorted_indices]
    A_Dev = A_Dev[sorted_indices]

    N_ppc = 100
    t_ppc = np.linspace(48, 144, N_ppc)
    
    return dict(t_Reg=t_Reg, A_Reg=A_Reg, N_Reg=len(t_Reg),
                t_Dev=t_Dev, A_Dev=A_Dev, N_Dev=len(t_Dev),
                  N_ppc=N_ppc, t_ppc=t_ppc)

stan_data = df_to_data('Development','Regeneration')

In [9]:
samples = sm.sample(data=stan_data,iter_warmup=3000,iter_sampling=1000,show_console = False )
samples = az.from_cmdstanpy(posterior=samples, posterior_predictive=['A_Dev_ppc', 'A_Reg_ppc'])
bebi103.stan.check_all_diagnostics(samples)

15:25:15 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

15:30:10 - cmdstanpy - INFO - CmdStan done processing.
15:30:10 - cmdstanpy - WARNING - Non-fatal error during sampling:
Exception: normal_lpdf: Location parameter[1] is -nan, but must be finite! (in 'fit_Dose.stan', line 103, column 4 to column 110)
	Exception: normal_lpdf: Location parameter[1] is -nan, but must be finite! (in 'fit_Dose.stan', line 103, column 4 to column 110)
	Exception: normal_lpdf: Location parameter[1] is -nan, but must be finite! (in 'fit_Dose.stan', line 103, column 4 to column 110)
	Exception: normal_lpdf: Location parameter[1] is -nan, but must be finite! (in 'fit_Dose.stan', line 103, column 4 to column 110)
	Exception: normal_lpdf: Location parameter[1] is -nan, but must be finite! (in 'fit_Dose.stan', line 103, column 4 to column 110)
	Exception: normal_lpdf: Location parameter[1] is -nan, but must be finite! (in 'fit_Dose.stan', line 103, column 4 to column 110)
	Exception: normal_lpdf: Location parameter[1] is -nan, but must be finite! (in 'fit_Dose.stan


Effective sample size looks reasonable for all parameters.

Rhat looks reasonable for all parameters.

87 of 4000 (2.175%) iterations ended with a divergence.
  Try running with larger adapt_delta to remove divergences.

0 of 4000 (0.0%) iterations saturated the maximum tree depth of 10.

E-BFMI indicated no pathological behavior.


4

In [12]:
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_0_Dev', 'g_0_Dev',  'C0', 'C1', 'g_max', 'tau_g'],
        xtick_label_orientation=np.pi / 4,
        plot_ecdf=True
    )
)


In [14]:
bokeh.io.show(
    bebi103.viz.corner(
        samples,
        parameters=['A_0_Reg', 'g_0_Reg', 'D_decrease', 'C0', 'C1', 'g_max', ],
        xtick_label_orientation=np.pi / 4,
        plot_ecdf=True
    )
)

In [10]:
samples

Inference data with groups:
	> posterior
	> posterior_predictive
	> sample_stats

In [16]:
output_file("area_DevREG_ppc_At_Dev.html")
f_ppc = samples.posterior_predictive.A_Dev_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Dev_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(stan_data["t_ppc"]),
        data=np.dstack((stan_data["t_Dev"], stan_data["A_Dev"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='A',
    )
)

In [17]:
output_file("area_DevREG_ppc_At_Reg.html")
f_ppc = samples.posterior_predictive.A_Reg_ppc.stack(
    {"sample": ("chain", "draw")}
).transpose("sample", "A_Reg_ppc_dim_0")

bokeh.io.show(
    bebi103.viz.predictive_regression(
        f_ppc,
        samples_x=np.array(stan_data["t_ppc"]),
        data=np.dstack((stan_data["t_Reg"], stan_data["A_Reg"])).squeeze(),
        #x_axis_type="log",
        x_axis_label='t',
        y_axis_label='A',
    )
)


In [18]:
posterior = samples.posterior

# Dictionary to store mean and confidence intervals for each variable
results ={}

# Loop through each variable in the posterior
for var_name in posterior.data_vars:
    data = posterior[var_name].values.flatten()
    
    results[var_name]=data

print(results)
# Convert the results dictionary to a pandas DataFrame
results_df = pd.DataFrame(results)

# Display the table
print(results_df)
Output_path=os.path.join(Sahra_Shivani_path,'sorted_data')
Curv_Thick_path=os.path.join(Output_path,'for_curv_thick')
results_df.to_csv(os.path.join(Curv_Thick_path,"area_sampled_parameter_results_Dose.csv"), index=False)

{'A_0_Dev': array([1.76518, 2.23439, 2.29929, ..., 1.63835, 1.89937, 1.97275]), 'A_0_Dev_tilde': array([0.246789, 0.349158, 0.361593, ..., 0.214406, 0.278611, 0.295073]), 'A_0_Reg': array([1.45008, 1.89514, 1.23463, ..., 1.53116, 1.80468, 1.54048]), 'A_0_Reg_tilde': array([0.161393 , 0.277642 , 0.0915353, ..., 0.18502  , 0.256401 ,
       0.187656 ]), 'C0': array([ 0.0748092 , -0.00073742,  0.323539  , ...,  0.0941505 ,
        0.22076   , -0.0473446 ]), 'C1': array([0.0959972, 0.0918332, 0.0369672, ..., 0.13646  , 0.0515964,
       0.132114 ]), 'D_0_Reg': array([-6.48037, -8.81765, -7.3559 , ..., -8.30219, -7.96876, -6.93675]), 'D_decrease': array([36.8756, 28.4653, 25.7066, ..., 58.1944, 26.3582, 37.3129]), 'D_decrease_tilde': array([1.56674, 1.45432, 1.41005, ..., 1.76488, 1.42092, 1.57186]), 'g_0_Dev': array([0.125546 , 0.0980333, 0.0104493, ..., 0.147721 , 0.11195  ,
       0.0936453]), 'g_0_Reg': array([ 0.0538351, -0.0307864,  0.022519 , ...,  0.0120298, -0.0380449,
        0.02

In [14]:
bebi103.stan.clean_cmdstan()